In [5]:
%cd ..
from llm_lora import QwenQACInference, load_model_qwen
from speedy import *

/anhvth5/projects/guidance


In [2]:
model, tok = load_model_qwen('/anhvth5/checkpoint-llm-finetuned/qwen_14B/full_010124/', device_map=0)
qac = QwenQACInference(model=model, tokenizer=tok)
print('QwenQACInference loaded')

Syncing /anhvth5/checkpoint-llm-finetuned/qwen_14B/full_010124 to /home/ubuntu/.cache/huggingface/local_hub/qwen_01fe265c0f718c13
sending incremental file list

sent 1,102 bytes  received 17 bytes  2,238.00 bytes/sec
total size is 28,337,880,096  speedup is 25,324,289.63


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

QwenQACInference loaded


In [8]:
nli_data = load_by_ext('../../data/chat-formated-dataset/tvpl_giaothong_rag_nli_vietnamese_train.json')
print('Example: ', nli_data[0])

Example:  {'conversations': [{'from': 'user', 'value': 'Please determine the relationship between the following premise and hypothesis:\nPremise: ```Question: Việc thông báo trục vớt tài sản chìm đắm gây nguy hiểm được thực hiện trong thời gian nào? \nAnswer: Theo Điều 279 Bộ luật Hàng hải Việt Nam 2015, chủ sở hữu tài sản chìm đắm gây nguy hiểm có nghĩa vụ thông báo ngay cho Giám đốc Cảng vụ hàng hải nơi gần nhất biết sự cố xảy ra và phải tiến hành trục vớt tài sản đó trong thời hạn theo quy định của Chính phủ.\n```\n\nHypothesis: ```Nghị định 05/2017/NĐ-CP xử lý tài sản chìm đắm đường thủy nội địa vùng nước cảng biển vùng biển Việt ...\nChương II |  Điều 13. Thủ tục phê duyệt phương án trục vớt tài sản chìm đắm\n3. Trường hợp hồ sơ nhận được không hợp lệ thì chậm nhất 02 ngày làm việc kể từ ngày nhận được hồ sơ, cơ quan có thẩm quyền phê duyệt phải hướng dẫn người nộp hoàn thiện hồ sơ theo quy định tại Nghị định này. Nếu hồ sơ hợp lệ thì trong thời hạn không quá 10 ngày kể từ ngày nh

In [16]:
NLI_TEMPLATE = '''Please determine the relationship between the following premise and hypothesis:
Premise: ```Question: {question}
Answer: {answer}
```
Hypothesis: ```{document}```'''

In [19]:
from guidance import *

In [20]:
model = models.QwenChat(model, tok)

2024-01-06 08:12:31.943 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


In [22]:
qac_test = load_by_ext('/anhvth5/data/qac-data/za_qac/qac_test.jsonl')
qac_test[0]

{'id': 'test_1001p1',
 'question': 'Con trai đầu của Ngô Quyền tên gì',
 'title': 'Nhà Ngô',
 'text': 'Ngô Quyền hạ thành Đại La , giết Công Tiễn rồi bày trận trên sông Bạch Đằng đón quân Nam Hán .',
 'label': False}

In [64]:
item = qac_test[33]
question = item['question']
answer = ''
document = '{} | {}'.format(item['title'], item['text'])

with assistant():
    lm = model + 'You are a helpful assistant'
    
with user():
    lm += NLI_TEMPLATE.format(
        question=question,
        answer=answer,
        document=document
    )
with assistant():
    lm += 'Mức độ liên quan (1-5)' + gen()

In [65]:
# out = qac(question, document, '5level', expected_answer='10 vị vua', force=True, decode=True)
# out